# Establishing a Connection - Carol
## Role: Invitee

If you want to start at Part 1 of this tutorial series go [here](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%201%20-%20Self-Sovereign%20Identity.ipynb).

In this notebook we'll be responding to an invitation from Alice through our Agents. This notebook has the following phases:

<b>Begin with the [Alice notebook](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb)</b>


6. Instatiate the controller for our Agent
7. Set up a message listener running as a service on our controller
8. Paste the invitation from the Alice notebook into the invitation variable here
9. Accept the invitation

<b>Carry on at the Alice side</b>
    
14. Check if connection active


### 6. Instatiate the controller for our Agent

In [6]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8082
WEBHOOK_BASE = ""
ADMIN_URL = "http://carol-agent:8081"

agent_controller = AriesAgentController(admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


In [7]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

### 7. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using PyPubSub. The default listeners are used to update state and print logs.


In [8]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Subscribing too: connections


### 8. Paste the invitation from the Alice notebook into the invitation variable here;

In [9]:
#Paste in invitation from Alice agent
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '9676a279-d978-42bf-9e51-757f038b36d4', 'label': 'Alice', 'serviceEndpoint': 'http://192.168.65.3:8020', 'recipientKeys': ['Bve459nP9qUssiivfn7MMjPpxp5RhNzgyFfTTE61db2e']}

### 9. Accept the invitation, then move to Alice's notebook

In [10]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


ClientResponseError: 401, message='Unauthorized', url=URL('http://carol-agent:8081/connections/receive-invitation')

Error during POST /connections/receive-invitation: 401, message='Unauthorized', url=URL('http://carol-agent:8081/connections/receive-invitation')


### 13. Check if connection state is active

It should be if you send a trust ping from Alice's side

In [11]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

Alice AGENT CONNECTION
{'connection_id': 'd98eaf8a-7b8f-4a32-b230-cecf0fb691af', 'their_did': 'L1s7TR6CiAMVpR2skQCbfU', 'request_id': '565d8c72-9295-42a1-b1b6-4bfd3a78e237', 'invitation_key': '2ZfKo22KWUudXnBq6WsCnHh2op3VkoCWyugHEd6WyCZn', 'their_role': 'inviter', 'my_did': 'Dr9aLJnoB8RpnWz1EDYDzy', 'updated_at': '2023-01-02 07:04:05.724173Z', 'routing_state': 'none', 'state': 'active', 'rfc23_state': 'completed', 'accept': 'manual', 'created_at': '2023-01-02 07:03:50.544096Z', 'their_label': 'Alice', 'invitation_mode': 'once'}
State: active


## Optional: Remove connection

In [8]:
# remove = await agent_controller.connections.remove_connection(alice_id)
print(remove)

{}


In [4]:
response = await agent_controller.connections.get_connections()
print(response)

ClientResponseError: 401, message='Unauthorized', url=URL('http://carol-agent:8081/connections')

Error during GET /connections: 401, message='Unauthorized', url=URL('http://carol-agent:8081/connections')


## End of Tutorial

#### Terminate Controller & Stop Webhook Server

**Note: You will need to run this command when combining this example with others such as Issuer**

In [5]:
response = await agent_controller.terminate()
print(response)

None
